# Helical Extension Spring

<table><tr>
<td>
<img src="./img/Helical_Extension_Spring/Extension_Spring_-_Figure_10-6.png" width=300"/> </td>
<td>
<img src="./img/Helical_Extension_Spring//Extension_Spring_-_Figure_10-7.png" width=550"/> </td>
</tr></table>

Importing the relevant Classes

In [2]:
from me_toolbox.springs import ExtensionSpring, Spring
from me_toolbox.tools import inch_to_millimetre as im
from me_toolbox.tools import millimetre_to_inch as mi
from me_toolbox.tools import lbs_to_newtons as pn
from me_toolbox.tools import lbs_per_in_to_newtons_per_mm as pinmm

## Geometric properties

d - Wire diameter<br>
OD - Spring outside diameter<br>
D - Spring mean diameter<br>
r1 - Hook's main radius<br>
r2 - Hook's secondary radius

In [3]:
d = 0.88
D = 5.41
r1 = 2.69
r2 = 2.672

## Material properties

Table 10-4 from Shigley:
<table><tr>
<td>
<img src="./img/Helical_Compression_Spring/Tensile_strength_estimation.png" width=360"/> </td>
<td>
<img src="./img/Helical_Extension_Spring/Mechanical_properties_of_some_spring_wires_-_table_10-5.png" width=300"/> </td>
</tr></table>

Material options for use in Spring.material_prop method are:<br>
  * music wire<br>
  * oq&t wire<br>
  * hard-drawn wire<br>
  * chrome-vanadium wire<br>
  * chrome-silicon wire<br>
  * 302 stainless wire<br>
  * phosphore-bronze wire<br>

G - Shear modulus<br>
E - Elastic modulus<br>
Sut - Ultimate tensile strength

In [4]:
# music wire
G = 80e3  # [MPa]
E = 197.9e3  # [MPa]
Sut = Spring.material_prop('hard-drawn wire', d, metric=True, verbose=False)
Sut = 1823.3
print(f"Sut = {Sut:.2f}")

Sut = 1823.30


Table 10-6 from Shigley:
<table><tr>
<td> <img src="./img/Helical_Extension_Spring/Present_of_tensile_strength_-_Table_10-7.png" width=600"/> </td>
</tr></table>


In [5]:
body_torsion_yield_percent = 0.45
end_torsion_yield_percent = 0.4
end_bending_yield_percent = 0.75

## Forces and spring rate

k - Spring rate<br>
Fmax - Maximum force<br>
Fi - Initial force (force at free state from manufacturing process)

In [6]:
k = 3.13  # [N/mm]
Fmax = 22.24  # [N]
Fi = 5.29  # [N]

## Optional parameters

Shot peening - a cold working process in which the surface of the spring is impacted with a small spherical mediacalled shot in order to increase the fatigue resistance.<br>
rho - Material density

In [7]:
peened=False
rho = 7800  # [kg/m^3]

## Defining the spring

Required parameters:
  * max_force - The maximum force in $[N]$
  * initial_tension - The initial tension in $[N]$
  * wire_diameter - The wire diameter in $[mm]$
  * spring_diameter - The spring diameter in $[mm]$
  * hook_r1 - The spring's hook big radius in $[mm]$
  * hook_r2 - The spring's hook small radius in $[mm]$
  * ultimate_tensile_strength - The ultimate tensile strength in $[MPa]$
  * body_shear_yield_precent - the precentage of the shear yield strength(Ssy) from the ultimate tensile strength (Sut)
  * hook_normal_yield_percent - the precentage of the normal yield strength(Sy) from the ultimate tensile strength (Sut)
  * hook_shear_yield_percent - the precentage of the shear yield strength(Ssy) from the ultimate tensile strength (Sut)
  * shear_modulus - The material shear modulus in $[MPa]$
  * elastic_modulus - The material elastic modulus in $[MPa]$
  * spring_rate - The spring rate in $[N/mm]$

Optional parameters: <br>
(the bold parameters are the defaults)
  * shot_peened=**False** - If the spring underwent shot peening
  * density=**None** - The Material density in $[kg/mm^3]$

In [8]:
spring = ExtensionSpring(max_force=Fmax, 
                         initial_tension=Fi,
                         wire_diameter=d, 
                         spring_diameter=D, 
                         hook_r1=r1, hook_r2=r2,
                         ultimate_tensile_strength=Sut, 
                         body_shear_yield_percent=body_torsion_yield_percent, 
                         hook_normal_yield_percent=end_bending_yield_percent, 
                         hook_shear_yield_percent=end_torsion_yield_percent,
                         shear_modulus=G, elastic_modulus=E,
                         spring_rate=k, 
                         shot_peened=peened, 
                         density=rho)

### Calculated geometric properties

In [9]:
print(f"Free Length (L0) = {spring.free_length:.2f}\n"
      f"Active Coils (Na) = {spring.active_coils:.2f}\n"
      f"Body Coils (Nb) = {spring.body_coils:.2f}")

Free Length (L0) = 20.09
Active Coils (Na) = 11.94
Body Coils (Nb) = 11.54


### Calculated material properties

In [10]:
print(f"Shear yield strength (Ssy) = {spring.shear_yield_strength:.2f}\n"
      f"Shear ultimate strength (Ssu) = {spring.shear_ultimate_strength:.2f}\n")

Shear yield strength (Ssy) = 820.49
Shear ultimate strength (Ssu) = 1221.61



## Static analysis

> Note: The verbose option can be used to print more information

In [11]:
ns = spring.static_safety_factor(verbose=False)
print(f"body safety factors: {ns['n_body']:.2f}\n"
      f"hook normal safety factors: {ns['n_hook_normal']:.2f}\n"
      f"hook shear safety factors: {ns['n_hook_shear']:.2f}")

body safety factors: 1.46
hook normal safety factors: 1.29
hook shear safety factors: 1.41


## Fatigue analysis
> Note: The verbose option can be used to print more information

In [12]:
F_max = 22.24  # [N]
F_min = 6.67
n = spring.fatigue_analysis(max_force=F_max, min_force=F_min, criterion='gerber', reliability=50, verbose=False)
print(f"The minimal safety factor is: {min(n, key=n.get)} = {n[min(n, key=n.get)]:.2f}")

The minimal safety factor is: nf_hook_normal = 1.05


### Fatigue safety factor according to Gerber failure criteria

In [13]:
print(f"Body's fatigue safety factor (nf) = {n['nf_body']:.2f}\n"
      f"Hook's normal fatigue safety factor (nf) = {n['nf_hook_normal']:.2f}\n"
      f"Hook's shear fatigue safety factor (nf) = {n['nf_hook_shear']:.2f}\n")

Body's fatigue safety factor (nf) = 1.19
Hook's normal fatigue safety factor (nf) = 1.05
Hook's shear fatigue safety factor (nf) = 1.29



### Static safety factor according to Langer failure criteria

In [14]:
print(f"body's static safety factor (ns) = {n['ns_body']:.2f}\n"
      f"Hook's normal static safety factor (ns) = {n['ns_hook_normal']:.2f}\n"
      f"Hook's shear Static safety factor (ns) = {n['ns_hook_shear']:.2f}\n")

body's static safety factor (ns) = 1.61
Hook's normal static safety factor (ns) = 1.29
Hook's shear Static safety factor (ns) = 1.41



## Frequency analysis

The natural frequency must be greater than 20 times the working frequency<br>
$(f_{natural} > 20 \cdot f_{working})$
> Note:
> - The verbose option can be used to print more information<br>
> - The methode returns a dictionary but the output is suppressed

In [18]:
spring.natural_frequency(density=rho, working_frequency=0.5, verbose=True);

Note: the natural frequency=0.91 for fixed ends is not larger than 20*working frequency=10.00 
which means the spring can resonance

Note: the natural frequency=0.45 for one fixed and one free ends is not larger than 20*working frequency=10.00 
which means the spring can resonance

